In [1]:
from pymongo import MongoClient
import json

In [2]:
john = MongoClient("")

In [6]:
db=john["Guvi"]
collection = db["Students"]

In [33]:
data = [json.loads(line) for line in open('students.json', 'r')]
collection.insert_many(data)

### 1)      Find the student name who scored maximum scores in all (exam, quiz and homework)?

In [8]:
# exam
a = collection.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'exam'}},
    {'$sort':{'scores.score':-1}},
    {'$project':{'_id':0,'name':1,'scores.score':1}},
    {'$limit':1}
])
for i in a:
    print(i)

{'name': 'Margart Vitello', 'scores': {'score': 99.33685767140612}}


In [9]:
# quiz
a = collection.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'quiz'}},
    {'$sort':{'scores.score':-1}},
    {'$project':{'_id':0,'name':1,'scores.score':1}},
    {'$limit':1}
])
for i in a:
    print(i)

{'name': 'Cody Strouth', 'scores': {'score': 99.80348240553108}}


In [12]:
# homework
a = collection.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'homework'}},
    {'$sort':{'scores.score':-1}},
    {'$project':{'_id':0,'name':1,'scores.score':1}},
    {'$limit':1}
])
for i in a:
    print(i)

{'name': 'Whitley Fears', 'scores': {'score': 99.77237745070993}}


### 2)      Find students who scored below average in the exam and pass mark is 40%?

In [13]:
a = collection.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'exam'}},
    {'$group':{'_id':'$scores.type','average':{'$avg':'$scores.score'}}},
    {'$project':{'_id':1,'average':1}},
])
for i in a:
    print(i)

{'_id': 'exam', 'average': 48.67367075950175}


In [14]:
a = collection.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'exam'}},
    {'$group':
    {
        '_id':'$scores.type',
        'average':{'$avg':'$scores.score'}
    }},
    {'$project':{'_id':0,'average':1}}
])
for i in a:
    average = i['average']
res = collection.find({'scores':
                {'$elemMatch':
                  {'$and':
                    [{'type':'exam'},{'score':{'$gte':40,'$lt':average}}]}}},
                   {'_id':0})
for i in res:
    print(i['name'])

Wilburn Spiess
Denisha Cast
Gisela Levin
Tressa Schwing
Rosana Vales
Tamika Schildgen
Gwyneth Garling
Kayce Kenyon
Terica Brugger
Chad Rahe
Jenise Mcguffie
Cassi Heal
Demarcus Audette
Flora Duell
Quincy Danaher
Alix Sherrill
Laureen Salomone
Gwyneth Garling
Aleida Elsass
Alica Pasley
Malisa Jeanes
Harriett Velarde
Mariela Sherer


### 3)      Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.

In [15]:
# exam
a = collection.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'exam'}},
    {'$project':{'_id':1,'name':1,'score':'$scores.score'}}
])
for i in a:
    if i['score'] >= 40:
        res = collection.update_one({'_id':i['_id']},{'$set':{'scores.0.status':'Pass'}})
    else:
        res = collection.update_one({'_id':i['_id']},{'$set':{'scores.0.status':'Fail'}})

In [17]:
# quiz
a = collection.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'quiz'}},
    {'$project':{'_id':1,'name':1,'score':'$scores.score'}}
])
for  i in a:
    if i['score'] >= 40:
        res = collection.update_one({'_id':i['_id']},{'$set':{'scores.1.status':'Pass'}})
    else:
        res = collection.update_one({'_id':i['_id']},{'$set':{'scores.1.status':'Fail'}})

In [18]:
# homework
a = collection.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'homework'}},
    {'$project':{'_id':1,'name':1,'score':'$scores.score'}}
])
for  i in a:
    if i['score'] >= 40:
        res = collection.update_one({'_id':i['_id']},{'$set':{'scores.2.status':'Pass'}})
    else:
        res = collection.update_one({'_id':i['_id']},{'$set':{'scores.2.status':'Fail'}})

### 4)       Find the total and average of the exam, quiz and homework and store them in a separate collection.

In [19]:
total_avg = db["test_1"]

In [24]:
a = collection.aggregate(
[
    {'$unwind':'$scores'},
    {'$group':
    {
        '_id':'$scores.type',
        'Average':{'$avg':'$scores.score'},
        'Sum' : {'$sum':'$scores.score'}
    }},
    {'$project':{'_id':1,'Average':1,'Sum':1}}
])
for i in a:
    print(i)
    res = total_avg.insert_one(i)
    print(res.acknowledged)

{'_id': 'exam', 'Average': 48.67367075950175, 'Sum': 9734.734151900351}
True
{'_id': 'homework', 'Average': 67.81869620661149, 'Sum': 13563.739241322297}
True
{'_id': 'quiz', 'Average': 48.99672319430254, 'Sum': 9799.344638860508}
True


### 5)      Create a new collection which consists of students who scored below average and above 40% in all the categories

In [31]:
below_avg = db["test_2"]

In [28]:
#exam
a = collection.aggregate(
[
    {'$unwind':'$scores'},
    {'$group':
    {
        '_id':'$scores.type',
        'average':{'$avg':'$scores.score'}
    }},
    {'$project':{'_id':1,'average':1}}
])
for i in a:
    if i['_id'] == 'exam':
        exam_avg = i['average']
    elif i['_id'] == 'homework':
        hw_avg = i['average']
    else:
        quiz_avg = i['average']
print('Exam Average = '+ str(exam_avg) + '\nHomeWork Average = '+ str(hw_avg) + '\nQuiz Average = ' + str(quiz_avg))

Exam Average = 48.67367075950175
HomeWork Average = 67.81869620661149
Quiz Average = 48.99672319430254


In [33]:
b = collection.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'exam','score':{'$gte':40,'$lt':exam_avg}}]},
    }
},{'_id':0})
count = 0
for i in b:
    res = below_avg.insert_one(i)
    if res.acknowledged == True:
        count = count + 1
print('Inserted Documents: '+ str(count))

Inserted Documents: 23


In [34]:
b = collection.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'quiz','score':{'$gte':40,'$lt':quiz_avg}}]},
    }
},{'_id':0})
count = 0
for i in b:
    res = below_avg.insert_one(i)
    if res.acknowledged == True:
        count = count + 1
print('Inserted Documents: '+ str(count))

Inserted Documents: 19


In [36]:
b = collection.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'homework','score':{'$gte':40,'$lt':hw_avg}}]},
    }
},{'_id':0})
count = 0
for i in b:
    res = below_avg.insert_one(i)
    if res.acknowledged == True:
        count = count + 1
print('Inserted Documents: '+ str(count))

Inserted Documents: 61


### 6)      Create a new collection which consists of students who scored below the fail mark in all the categories.

In [ ]:
failed = db["test_3"]

In [ ]:
result = collection.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'exam','score':{'$lt':40}},
                 {'type':'homework','score':{'$lt':40}},
                 {'type':'quiz','score':{'$lt':40}}]},
    }
},{'_id':0})
count = 0
for i in result:
    res = failed.insert_one(i)
    if res.acknowledged == True:
        count = count + 1
print('Inserted Documents: '+ str(count))

### 7)      Create a new collection which consists of students who scored above pass mark in all the categories

In [ ]:
passed = db["test_4"]

In [ ]:
result = collection.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'exam','score':{'$gt':40}},
                 {'type':'homework','score':{'$gt':40}},
                 {'type':'quiz','score':{'$gt':40}}]},
    }
},{'_id':0})
count = 0
for i in result:
    res = passed.insert_one(i)
    if res.acknowledged == True:
        count = count + 1
print('Inserted Documents: '+ str(count))